In [2]:
import os
import psycopg2
import pandas as pd
import numpy as np
import numba as nb
import pickle
import logging
from tqdm.notebook import tqdm
from datetime import datetime
import pytz
from scipy.interpolate import interp1d

import astropy
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy.time import Time
from astropy.table import Table
from astropy.timeseries import aggregate_downsample, TimeSeries
import astropy.units as u

data_dir = '../../py/desiforecast/data'

In [3]:
print(astropy.__version__)

5.0


In [22]:
tower_data = TimeSeries.read(f'{data_dir}/environmentmonitor_tower.fits')
dome_data = TimeSeries.read(f'{data_dir}/environmentmonitor_dome.fits')
telescope_data = TimeSeries.read(f'{data_dir}/environmentmonitor_telescope.fits')
seeing_data = TimeSeries.read(f'{data_dir}/etc_seeing.fits')
telemetry_data = TimeSeries.read(f'{data_dir}/etc_telemetry.fits')
tcs_data = TimeSeries.read(f'{data_dir}/tcs_info.fits')

In [ ]:
tower_data = TimeSeries.from_pandas(tower_data)
dome_data = TimeSeries.from_pandas(dome_data)
telescope_data = TimeSeries.from_pandas(telescope_data)
seeing_data = TimeSeries.from_pandas(seeing_data)
telemetry_data = TimeSeries.from_pandas(telemetry_data)
tcs_data = TimeSeries.from_pandas(tcs_data)

In [ ]:
tower_data.write(f'{data_dir}/environmentmonitor_tower.fits', overwrite=True)
dome_data.write(f'{data_dir}/environmentmonitor_dome.fits', overwrite=True)
telescope_data.write(f'{data_dir}/environmentmonitor_telescope.fits', overwrite=True)
seeing_data.write(f'{data_dir}/etc_seeing.fits', overwrite=True)
telemetrydata.write(f'{data_dir}/etc_telemetry.fits', overwrite=True)
tcs_data.write(f'{data_dir}/tcs_info.fits', overwrite=True)


In [28]:
print(type(telescope_data.data))

AttributeError: 'TimeSeries' object has no attribute 'data'

In [ ]:
print(telescope_data['air_temp']-telescope_data['mirror_temp'])
print(telescope_data['mirror_temp'])
print(telescope_data['air_temp'])


In [25]:
# def preprocess_data(data):
#     nans = np.isnan(y)
#     x = lambda z: z.nonzero()[0]
#     data[nans] = np.interp(x(nans), x(~nans), y[~nans])
#     data
#     if 'seeing' in data.colnames:
#         pass
#     else:
#         data = aggregate_downsample(data, time_bin_size=6*u.s)
#     return data
def preprocess_data(rows, columns):
    """Preprocesses DESI telemetry data contained within Pandas dataframe.
    
    Parameters
    ----------
    rows : np.ndarray
        Unprocessed data from the DESI telemetry database
    
    columns : list[str]
        column names that were queired from database
    
    Returns
    -------
    data : astropy.io.fits.BinTableHDU
        Resampled and interpolated DESI telemetry data
    """
    data = pd.DataFrame(rows, columns=columns)
    data.set_index('time_recorded', inplace=True)
    data.index = data.index.strftime('%Y-%m-%dT%H:%M:%S.%f')
    data = data.replace({pd.NA: np.nan})
    data.index.name = 'time'
    data.reset_index(inplace=True)
    data['time'] = data['time'].astype(str)
    return fits.table_to_hdu(Table.from_pandas(data)

In [ ]:
labels = {}
# labels['environmentmonitor_tower'] = ['time_recorded', 'temperature', 'pressure', 'humidity', 'wind_speed', 'wind_direction', 'dimm', 'dewpoint']
# labels['environmentmonitor_dome'] = ['time_recorded', 'dome_left_upper', 'dome_left_lower', 'dome_right_upper', 'dome_right_lower', 'dome_back_upper', 'dome_back_lower', 'dome_floor_ne', 'dome_floor_nw', 'dome_floor_s']
# labels['environmentmonitor_telescope'] = ['time_recorded', 'mirror_avg_temp', 'mirror_desired_temp', 'mirror_temp', 'mirror_cooling', 'air_temp', 'air_flow', 'air_dewpoint']
labels['etc_seeing'] = ['time_recorded', 'etc_seeing', 'seeing']
labels['etc_telemetry'] = ['time_recorded', 'seeing', 'transparency', 'skylevel']
labels['tcs_info'] = ['time_recorded', 'mirror_ready', 'airmass']

host = 'replicator-db.desi.lbl.gov'
database = 'desi_dev'
user = 'desi_reader'
password = 'censored'
port = '5432'
preprocess=True

for table, columns in tqdm(labels.items(), desc='Querying database tables...'):
    conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print(f'{table}')
    with conn:
        with conn.cursor() as cur:
            cur.execute(f'SELECT {", ".join(columns)} FROM {table};')
            rows = np.asarray(cur.fetchall())
            data = preprocess_data(rows=rows, columns=columns)
            data.writeto(f'{data_dir}/{table}.fits', overwrite=True)

In [ ]:
print(type(tower_data.time[0]))
print(tower_data)

In [ ]:
print(type(dome_data.time[0]))
print(dome_data)

In [ ]:
print(type(telescope_data.time[0]))
print(telescope_data)

In [ ]:
print(type(seeing_data.time[0]))
print(seeing_data)

In [ ]:
print(type(telemetry_data.time[0]))
print(telemetry_data)

In [ ]:
print(type(tcs_data.time[0]))
print(tcs_data)